In [ ]:
import os.path as osp
import os
import pandas as pd
from utils.utils import get_timetsamp_with_random
from utils.map_utils import get_game_info_with_G_eval
from utils.eval_gpt_utils import get_csv,eval_gpt_batch

In [ ]:
def eval_all(rst_dir,map_dir,save_path,model_name,task_name,eval_difficulty):
    
    eval_rst={}
    for idx,game_name in enumerate(os.listdir(rst_dir)):
#     for idx,game_name in enumerate(['planetfall','pentari','ludicorp','zork1','spellbrkr',
#     'ballyhoo','murdac','zenon','zork2','zork3',
#     '905','anchor','awaken','adventureland','afflicted',
#     'balances','curses','cutthroat','deephome','detective',
#     'dragon','enchanter','enter','gold','hhgg',
#     'hollywood','huntdark','infidel','inhumane','jewel',
#     'karn','library','loose','lostpig','lurking',
#     'moonlit','murdac','night','omniquest','partyfoul']):
        if game_name in ['outputs_diff','.gitattributes','.gitignore','.git']:
            continue
        print(f'handling {game_name}')
                
        G_eval,G,actions,locations,all2all,all_pairs,walkthrough=get_game_info_with_G_eval(map_dir,game_name)
#         if task_name=='pathgen':
#             rst=eval_rf_gpt_batch(game_name,rst_dir,G_eval,all2all,all_pairs,model_name=model_name,eval_difficulty=eval_difficulty)
#         elif task_name=='stepnav':
#             rst=eval_df_gpt_batch(game_name,rst_dir,G_eval,all2all,all_pairs,model_name=model_name,eval_difficulty=eval_difficulty)
        rst=eval_gpt_batch(game_name,rst_dir,G_eval,all2all,all_pairs,task_type=task_name,model_name=model_name,eval_difficulty=eval_difficulty)
        eval_rst[game_name]=rst
        print('-----------------------')
        
    if not os.path.exists(save_path):
        os.makedirs(save_path)
        
    timestamp=get_timetsamp_with_random()
    target_name=osp.join(save_path,f'result_{eval_difficulty}_{timestamp}.csv')


    get_csv(eval_rst,target_name)

    df = pd.read_csv(target_name)
    df = df.sort_values('name')
    average = df.mean()
    
    # Convert first column to object type
    df[df.columns[0]] = df[df.columns[0]].astype(object)

    # Append the average row and set the value of the first column to 'average'
    df.loc[len(df)] = ['average'] + list(average.values)

    # Replace all NA/NaN in df with 'NA'
    df.fillna('NA', inplace=True)
    
    
#     print(average.values)
    weighted_average=[]
    
    weighted_average.append(average.values[6]/average.values[8])
    weighted_average.append(average.values[7]/average.values[8])
    
    weighted_average.append(average.values[12]/average.values[14])
    weighted_average.append(average.values[9]/average.values[11])
    
    weighted_average.append(average.values[13]/average.values[14])
    weighted_average.append(average.values[10]/average.values[11])
    
    for i in range(len(list(average.values))-len(weighted_average)):
        weighted_average.append(0.0)


    df.loc[len(df)] = ['weighted_average'] + weighted_average
    
    df.to_csv(f"{osp.join(save_path,f'sorted_result_{eval_difficulty}_{timestamp}.csv')}", index=False)
    if osp.isfile(target_name):
        os.remove(target_name)
        print(f'The file "{target_name}" has been deleted.')
    else:
        print(f'The file "{target_name}" does not exist.')


In [ ]:
# notebook input
rst_dir='/share/data/mei-work/kangrui/github/mango/kangrui/data/claude-results'
map_dir='/share/data/mei-work/kangrui/github/mango/data/'
save_dir='/share/data/mei-work/kangrui/github/mango/kangrui/eval_results/claude_overall_0912/'
model_names=['claude-2','claude-instant-1']
eval_difficulties=['strict','loose']
task_names=["stepnav","pathgen"]

In [ ]:
for model_name in model_names:
    for task_name in task_names:
        for eval_difficulty in eval_difficulties:
            print(save_dir)
            print(model_name)
            print(task_name)
            save_path=osp.join(save_dir,model_name,task_name)
            eval_all(rst_dir,map_dir,save_path,model_name,task_name,eval_difficulty)

In [ ]:
# path="/share/data/mei-work/kangrui/github/mango/kangrui/results_overall_0626/gpt-3.5-turbo/pathgen/sorted_result_loose_2023-06-26-09-42-09_648.csv"

In [ ]:
# df = pd.read_csv(path)
# average=df.loc[len(df)-1]

# weighted_average=[]
# for i in range(7,16,3):
#     weighted_average.append(average.values[i]/average.values[i+2])
#     weighted_average.append(average.values[i+1]/average.values[i+2])
# for i in range(9):
#     weighted_average.append(0.0)

# df.loc[len(df)] = ['weighted_average'] + weighted_average
# df.to_csv(path,index=False)